# AutoML in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a pipeline with Regression AutoML task.

**Motivations** - This notebook explains how to use Regression AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, command, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.automl import classification, regression
from azure.ai.ml.entities._job.automl.tabular import TabularFeaturizationSettings
from azure.ai.ml.entities import Environment, PipelineJobSettings

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../../configuration.ipynb)

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

Found the config file in: /config.json


enable_node_public_ip: true
id: /subscriptions/6977e295-0d7c-4557-8e0b-26e2f6532103/resourceGroups/rg-mlws/providers/Microsoft.MachineLearningServices/workspaces/mlws01/computes/cpu-cluster
idle_time_before_scale_down: 120
location: eastus2
max_instances: 2
min_instances: 0
name: cpu-cluster
provisioning_state: Succeeded
size: Standard_DS11_v2
ssh_public_access_enabled: false
tier: dedicated
type: amlcompute



# 3. Basic pipeline job with regression task

## 3.1 Build pipeline

In [4]:
env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="./environment/preprocessing_env.yaml",
    name="pipeline-custom-environment",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'pipeline-custom-environment', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/6977e295-0d7c-4557-8e0b-26e2f6532103/resourceGroups/rg-mlws/providers/Microsoft.MachineLearningServices/workspaces/mlws01/environments/pipeline-custom-environment/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpuds11002/code/Users/pupanda/azureml/automl/automl-regression-house-pricing-in-pipeline', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7016b5e50760>, 'serialize': <msrest.serialization.Serializer object at 0x7016b5e50640>, 'v

In [5]:
# Define pipeline
@pipeline(
    description="AutoML Regression Pipeline",
)
def automl_regression(
    regression_train_data, regression_validation_data, regression_test_data
):
    # define command function for preprocessing the model
    preprocessing_command_func = command(
        inputs=dict(
            train_data=Input(type="mltable"),
            validation_data=Input(type="mltable"),
            test_data=Input(type="mltable"),
        ),
        outputs=dict(
            preprocessed_train_data=Output(type="mltable"),
            preprocessed_validation_data=Output(type="mltable"),
            preprocessed_test_data=Output(type="mltable"),
        ),
        code="./preprocess.py",
        command="python preprocess.py "
        + "--train_data ${{inputs.train_data}} "
        + "--validation_data ${{inputs.validation_data}} "
        + "--test_data ${{inputs.test_data}} "
        + "--preprocessed_train_data ${{outputs.preprocessed_train_data}} "
        + "--preprocessed_validation_data ${{outputs.preprocessed_validation_data}} "
        + "--preprocessed_test_data ${{outputs.preprocessed_test_data}}",
        environment="pipeline-custom-environment@latest",
    )
    preprocess_node = preprocessing_command_func(
        train_data=regression_train_data,
        validation_data=regression_validation_data,
        test_data=regression_test_data,
    )

    # define the AutoML regression task with AutoML function
    regression_node = regression(
        primary_metric="r2_score",
        target_column_name="SalePrice",
        training_data=preprocess_node.outputs.preprocessed_train_data,
        validation_data=preprocess_node.outputs.preprocessed_validation_data,
        test_data=preprocess_node.outputs.preprocessed_test_data,
        featurization=TabularFeaturizationSettings(mode="off"),
        # currently need to specify outputs "mlflow_model" explicitly to reference it in following nodes
        outputs={"best_model": Output(type="mlflow_model")},
    )
    # set limits & training
    regression_node.set_limits(max_trials=1, max_concurrent_trials=1)
    regression_node.set_training(
        enable_stack_ensemble=False, enable_vote_ensemble=False
    )

    # define command function for registering the model
    command_func = command(
        inputs=dict(
            model_input_path=Input(type="mlflow_model"),
            model_base_name="regression_example_model",
        ),
        code="./register.py",
        command="python register.py "
        + "--model_input_path ${{inputs.model_input_path}} "
        + "--model_base_name ${{inputs.model_base_name}}",
        environment="azureml://registries/azureml/environments/sklearn-1.5/labels/latest",
    )
    register_model = command_func(model_input_path=regression_node.outputs.best_model)


pipeline_regression = automl_regression(
    regression_train_data=Input(path="./training-mltable-folder/", type="mltable"),
    regression_validation_data=Input(
        path="./validation-mltable-folder/", type="mltable"
    ),
    regression_test_data=Input(path="./test-mltable-folder/", type="mltable"),
)

In [6]:
# set pipeline level settings
# pipeline_regression.settings.default_compute = "cpu-cluster"
# pipeline_regression.settings.force_rerun = True

# Set default_compute, resources values for k8s
# Set default_compute only to some cpu cluster if k8s not in picture
pipeline_regression.settings = PipelineJobSettings(
    default_compute="k8s-compute",
    resources={"instance_type": "mlinstance", "instance_count": 2},
    force_rerun=True
)

# 3.2 Submit pipeline job

In [7]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_regression, experiment_name="pipeline_samples"
)
pipeline_job

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
pathOnCompute is not a known attribute

Experiment,Name,Type,Status,Details Page
pipeline_samples,good_shampoo_bxn1mh7j7q,pipeline,NotStarted,Link to Azure Machine Learning studio


In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

In [ ]:
print(type(pipeline_regression))

In [ ]:
print(type(pipeline_job))

## Schedule the pipeline job

In [ ]:
from azure.ai.ml.entities import (
    JobSchedule,
    CronTrigger,
    RecurrenceTrigger,
    RecurrencePattern,
)
from azure.ai.ml.constants import TimeZone
from datetime import datetime, timezone

In [ ]:
schedule_name = "schedule_recurrence_daily"

# every day at 1:00 and 1:30 UTC
recurrence_trigger = RecurrenceTrigger(
    frequency="day",
    interval=1,
    schedule=RecurrencePattern(hours=1, minutes=[0, 30]),
    time_zone=TimeZone.UTC,
)


job_schedule = JobSchedule(
    name=schedule_name, 
    trigger=recurrence_trigger,
    create_job=pipeline_regression
)

In [ ]:
# Create the job schedule
job_schedule_result = ml_client.schedules.begin_create_or_update(
    schedule=job_schedule
).result()

print(job_schedule_result)

In [ ]:
# List schedules in a workspace
schedules = ml_client.schedules.list()
[s.name for s in schedules]

In [ ]:
# Check details of the schedule
created_schedule = ml_client.schedules.get(name=schedule_name)
[created_schedule.name, created_schedule.is_enabled]

In [ ]:
# Get the schedule
schedule_name = "schedule_recurrence_daily"
job_schedule = ml_client.schedules.get(name=schedule_name)

In [ ]:
# Update the existing job schedule (optional)
# every day at 5:00 and 5:30 UTC
recurrence_trigger = RecurrenceTrigger(
    frequency="day",
    interval=1,
    schedule=RecurrencePattern(hours=5, minutes=[0, 30]),
    time_zone=TimeZone.UTC,
)
job_schedule.trigger = recurrence_trigger
job_schedule = ml_client.schedules.begin_create_or_update(
    schedule=job_schedule
).result()

In [ ]:
# # Disable schedule
# job_schedule = ml_client.schedules.begin_disable(name=schedule_name).result()
# job_schedule.is_enabled

In [ ]:
# # Only disabled schedules can be deleted
# ml_client.schedules.begin_disable(name=schedule_name).result()
# ml_client.schedules.begin_delete(name=schedule_name).result()